In [7]:
import numpy as np
from dataclasses import dataclass

In [8]:
class Sperm2D:
    def __init__(self,
                 length=70.1,
                 n_segments=50,
                 bending_modulus=1800,
                 amplitude=0.2,
                 wavenumber=1.0, # Note: not the conventional wavenumner. 2*pi*k*l/L, where k is the wavenumber
                 frequency=10.0,
                init_position=[0, 0], # tip of head
                init_angle=0,
                phase=0,
                head_semi_major=3,
                head_semi_minor=1):
        """
        Initialize a single 2D sperm filament.
        
        Parameters
        ----------
        length : float
            Total length of the filament (L).
        n_segments : int
            Number of discrete segments (N).
        bending_modulus : float
            Bending stiffness K_B.
        amplitude, wavenumber, frequency : float
            Parameters for the preferred-curvature waveform kappa(s,t).
        """
        # Geometry
        self.L = length
        self.N = n_segments
        self.Delta_L = length / n_segments
        self.a = head_semi_major
        self.b = head_semi_minor
        
        # Mechanical parameters
        self.K_B = bending_modulus
        self.K_0 = amplitude
        self.k = wavenumber
        self.omega = frequency
        self.phi = phase
        
        # State: positions Y[0..N], angles theta[0..N]
        self.Y_0 = init_position
        self.theta_0 = init_angle
        segments = np.hstack([self.a, np.arange(self.Delta_L/2+2.1*self.a, self.L+2.1*self.a, self.Delta_L)]) # Extra 0.1a to account for the linkage between the flagellum and the head
        self.Y = np.vstack([self.Y_0[0]+segments*np.cos(self.theta_0), self.Y_0[1]+segments*np.sin(self.theta_0)]).T # An array of length N+1 (midpoint of segmenets)
        self.theta = np.array([init_angle]*(self.N+1)) # An array of length N+1 (midpoint of segments)
        
        # Lagrange multipliers for constraints (N+2 of them), corresponding to the edges of each segment
        self.Lambda = np.zeros(self.N+2)
    
    def preferred_curvature(self, t):
        """
        Traveling-wave preferred curvature Kappa(s,t) along filament using Eq. 2.1 of Schoeller et al. 2018
        Returns an array of length N, corresponding to the midpoint of each filament segment only, i.e. no head.
        """
        s = np.arange(self.Delta_L/2, self.L, self.Delta_L)
        base = self.K_0 * np.sin(2*np.pi*self.k*s/self.L - self.omega*t)
        decay = np.where(s > self.L/2, 2*(self.L - s)/self.L, 1.0)
        kappa = base * decay
        return kappa
    
    def internal_moment(self, t):
        """
        Compute M_{n+1/2} for n=1..N+1 using Eq. 34 of Schoeller et al. 2020
        Returns an array of length N+2, corresponding to the edges of each segment.
        """
        kappa = self.preferred_curvature(t)
        t_hat_x, t_hat_y = np.cos(self.theta), np.sin(self.theta)
        cross = t_hat_x[:-1]*t_hat_y[1:] - t_hat_y[:-1]*t_hat_x[1:]
        delta_s = np.zeros(self.N)
        delta_s[0] = 1.1*self.a+self.Delta_L/2
        delta_s[1:] = self.Delta_L
        M = np.zeros(self.N+2)
        M[1:self.N+1] = self.K_B * (cross/delta_s - kappa)
        return M

In [9]:
sperm_1 = Sperm2D()

Not considering head. Abandoned

In [ ]:
def barrier_force(positions, a, b, F_S, chi=1.1):
    """
    Compute steric barrier forces for a set of segment positions. 
    Assume repulsion due to head-on approach involving filament is negligible.
    
    Parameters
    ----------
    positions : ndarray, shape (M, dim)
        Coordinates of M segments (e.g., 2D or 3D).
    a, b : floats
        Head and segment radii respectively
    F_S : float
        Reference strength of repulsion at contact.
    chi : float, optional
        Range factor for the barrier (default: 1.1).
    
    Returns
    -------
    forces : ndarray, shape (M, dim)
        Steric barrier forces on each segment.
    """
    M, dim = positions.shape
    forces = np.zeros_like(positions)
    r_cut = 2 * chi * a
    
    # Precompute denominator
    denom = 4 * a**2 * (chi**2 - 1)
    
    if exclude_pairs is None:
        exclude_pairs = set()
    
    for i in range(M):
        for j in range(i+1, M):
            if (i, j) in exclude_pairs or (j, i) in exclude_pairs:
                continue
            
            # vector separation and distance
            r_vec = positions[i] - positions[j]
            r = np.linalg.norm(r_vec)
            
            # apply barrier only if within cutoff
            if r < r_cut and r > 0:
                factor = (4 * a**2 * chi**2 - r**2) / denom
                # barrier scalar factor raised to the fourth power
                scalar = F_S * factor**4 / (2 * a)
                # force direction: repulsive, so along r_vec
                f = scalar * (r_vec / r)
                forces[i] += f
                forces[j] -= f  # equal and opposite
    return forces

# Example usage
# Define random positions of 10 segments in 2D
positions = np.random.rand(10, 2)
forces = compute_barrier_forces(positions, a=0.5, F_S=100.0)

In [ ]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Segment:
    id: int            # unique integer identifier
    kind: str          # 'head' or 'tail'
    length: float      # Δs for tail, 2a for head–tail link if you like
    radius: float      # a for tail, a_head for head
    midpoint: np.ndarray  # 2‑vector of current Y
    theta: float       # current orientation angle

class Sperm2D:
    def __init__(self, L=70.1, N=50, a_tail=0.5, a_head=3.0, **kwargs):
        self.L = L
        self.N = N
        self.ds = L/N
        self.a_tail = a_tail
        self.a_head = a_head

        # Build segment objects, ID 0 = head, 1..N = tail segments
        self.segments = []
        # head at index 0
        head_mid = np.array([0.0, 0.0])  # or init_position + a_head*unit
        head_theta = 0.0  # init_angle
        self.segments.append(
            Segment(id=0, kind='head',
                    length=2*a_head, radius=a_head,
                    midpoint=head_mid, theta=head_theta)
        )
        # tail segments indices 1..N
        for i in range(1, N+1):
            mid = np.array([2*a_head + (i-1)*self.ds + self.ds/2, 0.0])
            self.segments.append(
                Segment(id=i, kind='tail',
                        length=self.ds, radius=self.a_tail,
                        midpoint=mid, theta=head_theta)
            )
        # now you have N+1 segments

    def update_midpoints_and_thetas(self, Y_array, theta_array):
        """After solving, write back into each Segment object."""
        for seg, Y, th in zip(self.segments, Y_array, theta_array):
            seg.midpoint = Y
            seg.theta = th

    def compute_barrier_forces(self, F_S, chi=1.1):
        """
        Build capsule list then use segment-segment minimal-distance
        to get repulsion, where each segment uses its own radius.
        """
        M = len(self.segments)
        forces = np.zeros((M, 2))
        r_cut = lambda seg_i, seg_j: 2 * chi * max(seg_i.radius, seg_j.radius)

        # Precompute endpoints for each segment
        endpoints = []
        for seg in self.segments:
            t = np.array([np.cos(seg.theta), np.sin(seg.theta)])
            P = seg.midpoint - 0.5*seg.length*t
            Q = seg.midpoint + 0.5*seg.length*t
            endpoints.append((P, Q))

        # Now pairwise
        for i in range(M):
            for j in range(i+1, M):
                seg_i, seg_j = self.segments[i], self.segments[j]
                P, Q = endpoints[i]
                R, S = endpoints[j]
                # minimal distance
                d, Pi, Pj = segment_segment_min_dist(P, Q, R, S)
                cutoff = r_cut(seg_i, seg_j)
                if 0 < d < cutoff:
                    n = (Pi - Pj) / d
                    phi = (cutoff - d) / (cutoff - 2*max(seg_i.radius, seg_j.radius))
                    mag = F_S * phi**4
                    f = mag * n
                    forces[i] +=  f
                    forces[j] -=  f

        return forces